In [29]:
import tkinter as tk
from tkinter import messagebox

# --- HILFSFUNKTIONEN UND KLASSEN ---

class Spielstein:
    """Repräsentiert ein Haus, Kirchenschiff oder Kirchturm eines Spielers."""

    def __init__(self, spieler_id, typ, ausrichtung='N'):
        self.spieler = spieler_id  # 1 oder 2
        self.typ = typ             # Haus, Schiff, Turm
        self.ausrichtung = ausrichtung

    def drehen(self):
        """Dreht den Stein nach einem Zug um 90 Grad im Uhrzeigersinn."""
        richtungen = ['N', 'O', 'S', 'W']
        idx = richtungen.index(self.ausrichtung)
        self.ausrichtung = richtungen[(idx + 1) % 4]

    def get_farbe(self):
        return 'red' if self.spieler == 1 else 'blue'


def get_bewegungs_vektor(ausrichtung):
    if ausrichtung == 'N': return (0, -1)
    if ausrichtung == 'O': return (1, 0)
    if ausrichtung == 'S': return (0, 1)
    if ausrichtung == 'W': return (-1, 0)
    return (0, 0)


# --- KONSTANTEN ---
ROWS, COLS = 7, 7
CELL_SIZE = 80
WINDOW_SIZE = ROWS * CELL_SIZE
LINE_WIDTH = 3
OFFSET = 20


# --- HAUPTKLASSE SPIELBRETT ---

class Spielbrett(tk.Canvas):

    def __init__(self, master):
        super().__init__(
            master,
            width=WINDOW_SIZE + 2 * OFFSET,
            height=WINDOW_SIZE + 2 * OFFSET,
            bg='white'
        )
        self.pack()

        self.spielfeld_matrix = [[None] * COLS for _ in range(ROWS)]

        self.unplatzierte_steine = {
            1: self._initialisiere_steine(1, 'N'),
            2: self._initialisiere_steine(2, 'S')
        }

        self.pfarrer_pos = (4, 4)
        self.aktueller_spieler = 1
        self.ausgewaehlter_stein_pos = None
        self.spieler_farben = {1: 'red', 2: 'blue'}

        self.draw_grid()
        self.draw_labels()
        self.initialisiere_spielfeld()

        self.bind("<Button-1>", self.handle_click)
        master.title("Lass die Kirche im Dorf | Spieler 1")

    # ---------------- SETUP ----------------

    def _initialisiere_steine(self, spieler_id, standard_ausrichtung):
        steine = [Spielstein(spieler_id, 'Haus', standard_ausrichtung) for _ in range(7)]
        steine.append(Spielstein(spieler_id, 'Schiff', standard_ausrichtung))
        steine.append(Spielstein(spieler_id, 'Turm', standard_ausrichtung))
        return steine

    def initialisiere_spielfeld(self):
        self.draw_pfarrer(*self.pfarrer_pos)

        turm_p1 = self._hole_stein(1, 'Turm')
        turm_p1.ausrichtung = 'S'
        self.platziere_stein(1, 1, turm_p1)

        schiff_p1 = self._hole_stein(1, 'Schiff')
        schiff_p1.ausrichtung = 'N'
        self.platziere_stein(7, 7, schiff_p1)

        turm_p2 = self._hole_stein(2, 'Turm')
        turm_p2.ausrichtung = 'W'
        self.platziere_stein(1, 7, turm_p2)

        schiff_p2 = self._hole_stein(2, 'Schiff')
        schiff_p2.ausrichtung = 'N'
        self.platziere_stein(7, 1, schiff_p2)

    def _hole_stein(self, spieler, typ):
        stein = next(s for s in self.unplatzierte_steine[spieler] if s.typ == typ)
        self.unplatzierte_steine[spieler].remove(stein)
        return stein

    # ---------------- ZEICHNEN ----------------

    def draw_grid(self):
        for i in range(ROWS + 1):
            pos = i * CELL_SIZE + OFFSET
            self.create_line(OFFSET, pos, WINDOW_SIZE + OFFSET, pos, width=LINE_WIDTH)
            self.create_line(pos, OFFSET, pos, WINDOW_SIZE + OFFSET, width=LINE_WIDTH)

    def draw_labels(self):
        for j in range(COLS):
            x = (j + 0.5) * CELL_SIZE + OFFSET
            self.create_text(x, OFFSET / 2, text=str(j + 1))
        for i in range(ROWS):
            y = (i + 0.5) * CELL_SIZE + OFFSET
            self.create_text(OFFSET / 2, y, text=str(i + 1))

    def get_canvas_coords(self, row, col):
        return (
            (col - 0.5) * CELL_SIZE + OFFSET,
            (row - 0.5) * CELL_SIZE + OFFSET
        )

    def draw_pfarrer(self, row, col):
        x, y = self.get_canvas_coords(row, col)
        r = CELL_SIZE * 0.15
        self.create_oval(x - r, y - r, x + r, y + r, fill='black')
        self.create_text(x, y, text='P', fill='white')

    def draw_piece(self, row, col, stein):
        x, y = self.get_canvas_coords(row, col)
        farbe = stein.get_farbe()

        size = CELL_SIZE * 0.3
        self.create_rectangle(
            x - size, y - size,
            x + size, y + size,
            fill=farbe,
            tags=f"piece_{row}_{col}"
        )

        dx, dy = get_bewegungs_vektor(stein.ausrichtung)
        self.create_line(
            x, y,
            x + dx * size, y + dy * size,
            arrow=tk.LAST,
            width=3,
            tags=f"arrow_{row}_{col}"
        )

        self.create_text(x, y, text=stein.typ[0] + str(stein.spieler),
                         fill='white', tags=f"label_{row}_{col}")

    # ---------------- SPIELLOGIK ----------------

    def platziere_stein(self, row, col, stein):
        self.spielfeld_matrix[row - 1][col - 1] = stein
        self.draw_piece(row, col, stein)

    def get_grid_coords(self, x, y):
        if x < OFFSET or y < OFFSET:
            return None, None
        col = int((x - OFFSET) // CELL_SIZE) + 1
        row = int((y - OFFSET) // CELL_SIZE) + 1
        if 1 <= row <= ROWS and 1 <= col <= COLS:
            return row, col
        return None, None

    def handle_click(self, event):
        row, col = self.get_grid_coords(event.x, event.y)
        if row is None:
            return

        stein = self.spielfeld_matrix[row - 1][col - 1]

        if self.ausgewaehlter_stein_pos:
            self.ausgewaehlter_stein_pos = None
            self.wechsle_spieler()
        elif stein and stein.spieler == self.aktueller_spieler:
            self.ausgewaehlter_stein_pos = (row, col)

    def wechsle_spieler(self):
        self.aktueller_spieler = 3 - self.aktueller_spieler
        root.title(f"Lass die Kirche im Dorf | Spieler {self.aktueller_spieler}")


# --- PROGRAMMSTART ---
if __name__ == "__main__":
    root = tk.Tk()
    spiel = Spielbrett(root)
    root.mainloop()
